# Model analysis

#### Required libraries

In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

import warnings
warnings.filterwarnings("ignore")

### Data preparation

In [2]:
data = pd.read_csv('../data/extension/csv/data.csv')
data.replace('?', np.NAN, inplace=True)
data[data.columns.drop(['year', 'X65'])] = data[data.columns.drop(['year', 'X65'])].astype(float)
data[['year', 'X65']] = data[['year', 'X65']].astype('category')
    

useless_col = data.isna().sum()[data.isna().sum() > 5000].index
data.drop(useless_col, axis=1, inplace=True)
useless_row = data.isna().sum(axis=1)[data.isna().sum(axis=1) > 10].index
data.drop(useless_row, axis=0, inplace=True)
data.fillna(data.mean(), inplace=True)
data = data.sample(frac=1)
data.shape, useless_col

((43184, 64), Index(['X21', 'X37'], dtype='object'))

#### Train data and test data

To get a balanced dataset that is fairly dependant on each year :

In [3]:
train_ = pd.DataFrame()
test_ = pd.DataFrame()
for year in data.year.unique():
    tr, ts = train_test_split(data[data.year == year], test_size=0.2, random_state=9)
    train_ = train_.append(tr, ignore_index=True)
    test_ = test_.append(ts, ignore_index=True)

train_.shape, test_.shape

((34545, 64), (8639, 64))

#### Data standardization

In [4]:
categ = ['year', 'X65']

scaler =  StandardScaler()
scaler.fit(train_.drop(columns=categ))

train = pd.DataFrame(scaler.transform(train_.drop(columns=categ)), 
                      columns=train_.columns.drop(categ))
test = pd.DataFrame(scaler.transform(test_.drop(columns=categ)), 
                     columns=test_.columns.drop(categ))

train = pd.concat([train, train_[categ]], axis=1)
test = pd.concat([test, test_[categ]], axis=1)

In [5]:
train_y = train.pop('X65')
train_x = train.copy()

test_y = test.pop('X65')
test_x = test.copy()

#### Save scaler used for standardize train set

In [6]:
with open('saved_model/scaler.sav', 'wb') as f:
    pickle.dump(scaler, f)

#### Encode categorical variable : year

In [7]:
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

train_x.shape, test_x.shape

((34545, 67), (8639, 67))

### Model testing

12 max features because in visualisation part, we saw that only 12 was needed to reach the biggest parts of information contained in dataset

#### Logistic regression

In [219]:
lr = LogisticRegression()
cross_val_score(lr, train_x, train_y, cv=5)

array([0.9510783 , 0.95078883, 0.95035461, 0.9510783 , 0.9510783 ])

In [235]:
params = dict(solver=['newton-cg', 'sag'], C=[1, 2, 3])
lr = LogisticRegression(n_jobs=-1)
grid = GridSearchCV(lr, params, cv=3, n_jobs=-1)
grid.fit(train_x, train_y)
grid.cv_results_
print(lr)
print("\n".join([f"for {x} => mean : {y}" for x,y in zip(
                     grid.cv_results_['params'], grid.cv_results_['mean_test_score']
                 )]))
print(grid.best_estimator_)

LogisticRegression(n_jobs=-1)
for {'C': 1, 'solver': 'newton-cg'} => mean : 0.9497467071935156
for {'C': 1, 'solver': 'sag'} => mean : 0.9509335649153279
LogisticRegression(C=1, n_jobs=-1, solver='sag')


#### Linear Discriminant Analysis

In [236]:
lda = LDA(solver='svd', tol=1e-4)
cross_val_score(lda, train_x, train_y, cv=5)

array([0.94905196, 0.94992039, 0.94977565, 0.95049935, 0.95006513])

In [237]:
params = dict(solver=['svd', 'lsqr'], tol=[1e-3, 1e-4, 1e-5])
lda = LDA()
grid = GridSearchCV(lda, params, cv=5, n_jobs=-1)
grid.fit(train_x, train_y)
grid.cv_results_
print(lda)
print("\n".join([f"for {x} => mean : {y}" 
                 for x,y in zip(
                     grid.cv_results_['params'], 
                     grid.cv_results_['mean_test_score'])]
               ))
print(grid.best_estimator_)

LinearDiscriminantAnalysis()
for {'solver': 'svd', 'tol': 0.001} => mean : 0.9499203936893906
for {'solver': 'svd', 'tol': 0.0001} => mean : 0.9498624981907657
for {'solver': 'svd', 'tol': 1e-05} => mean : 0.9498335504414532
for {'solver': 'lsqr', 'tol': 0.001} => mean : 0.7708496164423215
for {'solver': 'lsqr', 'tol': 0.0001} => mean : 0.7708496164423215
for {'solver': 'lsqr', 'tol': 1e-05} => mean : 0.7708496164423215
LinearDiscriminantAnalysis(tol=0.001)


#### Random Forest

In [230]:
rf = RandomForestClassifier(n_jobs=-1, random_state=9)
cross_val_score(rf, train_x, train_y, cv=3)

array([0.96491533, 0.96552323, 0.96439427])

In [8]:
params = dict(criterion=['gini', 'entropy'], max_depth=[15, 35, 50])
rf = RandomForestClassifier(n_jobs=-1, random_state=9)

grid = GridSearchCV(rf, params, cv=3, n_jobs=-1)
grid.fit(train_x, train_y)
grid.cv_results_
print(rf)
print("\n".join([f"for {x} => mean : {y}" 
                 for x,y in zip(
                     grid.cv_results_['params'], 
                     grid.cv_results_['mean_test_score'])]
               ))
print(grid.best_estimator_)

RandomForestClassifier(n_jobs=-1, random_state=9)
for {'criterion': 'gini', 'max_depth': 15} => mean : 0.9654653350701983
for {'criterion': 'gini', 'max_depth': 35} => mean : 0.9658995513098857
for {'criterion': 'gini', 'max_depth': 50} => mean : 0.9658127080619482
for {'criterion': 'entropy', 'max_depth': 15} => mean : 0.9642784773483862
for {'criterion': 'entropy', 'max_depth': 35} => mean : 0.9638442611086987
for {'criterion': 'entropy', 'max_depth': 50} => mean : 0.9638442611086987
RandomForestClassifier(max_depth=35, n_jobs=-1, random_state=9)


#### Gradient Boosting

In [ ]:
gbm = GradientBoostingClassifier()
cross_val_score(gbm, train_x, train_y, cv=5)

In [ ]:
params = dict(loss=['deviance', 'exponential'], 
              learning_rate=[0.6, 0.1, 5e-3], 
              n_estimators=[100, 300], max_depth=[3,5,7])
gbm = GradientBoostingClassifier(random_state=9)

grid = GridSearchCV(gbm, params, cv=3, n_jobs=-1)
grid.fit(train_x, train_y)
grid.cv_results_
print(gbm)
print("\n".join([f"for {x} => mean : {y}" 
                 for x,y in zip(
                     grid.cv_results_['params'], 
                     grid.cv_results_['mean_test_score'])]
               ))
print(grid.best_estimator_)

#### SVM

In [ ]:
svm = SVC()
cross_val_score(svm, train_x, train_y, cv=5)

In [ ]:
params = dict(C=[1, 2, 3, 4], 
              kernel=['linear', 'rbf'],
              tol=[1e-3, 1e-4])
svm = SVC(random_state=9)

grid = GridSearchCV(svm, params, cv=3, n_jobs=-1)
grid.fit(train_x, train_y)
grid.cv_results_
print(svm)
print("\n".join([f"for {x} => mean : {y}" 
                 for x,y in zip(
                     grid.cv_results_['params'], 
                     grid.cv_results_['mean_test_score'])]
               ))
print(grid.best_estimator_)

#### Bonus neural network : MultiLayer Perceptron

In [ ]:
mlp = MLPClassifier()
mlp.fit(train_x, train_y)
mlp.score(train_x, train_y)

In [ ]:
params = dict(hidden_layer_sizes=[50, 100, 120, 150], 
              activation=['relu', 'tanh'], batch_size=[20, 50, 200])
mlp = MLPClassifier(random_state=9)

grid = GridSearchCV(mlp, params, cv=3, n_jobs=-1)
grid.fit(train_x, train_y)
grid.cv_results_
print(mlp)
print("\n".join([f"for {x} => mean : {y}" 
                 for x,y in zip(
                     grid.cv_results_['params'], 
                     grid.cv_results_['mean_test_score'])]
               ))
print(grid.best_estimator_)

### Underlying variables

In [114]:
with open('../data/udata_description.json', 'r') as f:
    udesc = json.loads(f.read())
    udesc['U35'] = 'bankrupt'

In [147]:
udata = pd.read_csv('../data/extension/csv/udata.csv')
udata.head()
udata.replace(np.inf, np.NAN, inplace=True)
useless_col = udata.isna().sum()[udata.isna().sum() > 5000].index
udata.drop(useless_col, axis=1, inplace=True)
useless_row = udata.isna().sum(axis=1)[udata.isna().sum(axis=1) > 5].index
udata.drop(useless_row, axis=0, inplace=True)
udata.fillna(udata.mean(), inplace=True)
udata.shape, [udesc[var] for var in useless_col]

((43246, 33), ['financial expenses', 'receivables', 'sales (n) / sales (n-1)'])

In [148]:
train_ = pd.DataFrame()
test_ = pd.DataFrame()
for year in udata.year.unique():
    tr, ts = train_test_split(udata[udata.year == year], test_size=0.2, random_state=9)
    train_ = train_.append(tr, ignore_index=True)
    test_ = test_.append(ts, ignore_index=True)

categ = ['year', 'U35']

uscaler =  StandardScaler()
uscaler.fit(train_.drop(columns=categ))

train = pd.DataFrame(uscaler.transform(train_.drop(columns=categ)), 
                      columns=train_.columns.drop(categ))
test = pd.DataFrame(uscaler.transform(test_.drop(columns=categ)), 
                     columns=test_.columns.drop(categ))

#### Save scaler used for standardize train set

In [149]:
with open('saved_model/uscaler.sav', 'wb') as f:
    pickle.dump(uscaler, f)

In [150]:
train = pd.concat([train, train_[categ]], axis=1)
test = pd.concat([test, test_[categ]], axis=1)
train_y = train.pop('U35')
train_x = train.copy()
test_y = test.pop('U35')
test_x = test.copy()

train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

train_x.shape, train_y.shape

((34594, 36), (34594,))

### Test

#### On logistic regression

In [151]:
lr = LogisticRegression()
cross_val_score(lr, train_x, train_y, cv=5)

array([0.95129354, 0.94999277, 0.95100448, 0.95129354, 0.9524429 ])

it appears that there is a tiny increasing with this dataset of underlying variable, let's test it on 
our best model, the MLP 